# Create the path variable

In [1]:
WORKSPACE_PATH = 'Documents/Tensorflow/workspace'
SCRIPTS_PATH = 'Documents/Tensorflow/scripts'
APIMODEL_PATH = 'Documents/Tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
CONFIG_PATH = MODEL_PATH+'/my_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'/my_ssd_mobnet/'

In [2]:
#import tensorflow module
import tensorflow as tf

# Create Label Map
this code will create annotation file in directory

In [3]:
labels = [{'name':'person', 'id':1}]

with open(ANNOTATION_PATH + '\label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# Create TF Records
generate the record for training and testing using tfrecord as generator based label_map annotation as label

In [7]:
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/train'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/train.record'}
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/test'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/test.record'}


Successfully created the TFRecord file: Documents/Tensorflow/workspace/annotations/train.record
Successfully created the TFRecord file: Documents/Tensorflow/workspace/annotations/test.record


# Download TF Models Pretrained Models

Download the SSD Mobnet architecture as pretrained models, then move it into pretrained_model_path directory

In [8]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

--2023-06-05 21:04:54--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 172.217.194.128
Connecting to download.tensorflow.org (download.tensorflow.org)|172.217.194.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20515344 (20M) [application/x-tar]
Saving to: 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'

     0K .......... .......... .......... .......... ..........  0%  756K 26s
    50K .......... .......... .......... .......... ..........  0% 1.36M 20s
   100K .......... .......... .......... .......... ..........  0% 1.88M 17s
   150K .......... .......... .......... .......... ..........  0% 2.29M 15s
   200K .......... .......... .......... .......... ..........  1% 1.66M 14s
   250K .......... .......... .......... .......... ..........  1% 1.35M 14s
   300K .......... .......... .......... .......... ......

## Clone from Tensorflow GitHub

In [ ]:
!cd Tensorflow && git clone https://github.com/tensorflow/models

# Copy Model Config to Training Folder

In [5]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet'

In [4]:
!mkdir {'Tensorflow\\workspace\\models\\'+CUSTOM_MODEL_NAME}
!copy {PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config'} {MODEL_PATH+'/'+CUSTOM_MODEL_NAME}


A subdirectory or file Tensorflow\workspace\models\my_ssd_mobnet already exists.


The syntax of the command is incorrect.


# Update Config For Transfer Learning

In [6]:
import tensorflow as tf
import object_detection
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [7]:
CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/pipeline.config'

In [8]:
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

In [9]:
config

{'model': ssd {
   num_classes: 1
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.009999999776482582
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.996999979019165
         scale: true
         epsilon: 0.0010000000474974513
       }
     }
     use_depthwise: true
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
       additional_layer_depth: 128
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   match

In [10]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)

In [9]:
pipeline_config.model.ssd.num_classes = 1
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/test.record']


In [25]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)

# Train the model

This cell will construct the command for doing training.
The command can be runned in the terminal

In [28]:
print("""python {}/research/object_detection/model_main_tf2.py --model_dir={}/{} --pipeline_config_path={}/{}/pipeline.config --num_train_steps=2000""".format(APIMODEL_PATH, MODEL_PATH,CUSTOM_MODEL_NAME,MODEL_PATH,CUSTOM_MODEL_NAME))


python Documents/Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Documents/Tensorflow/workspace/models/my_ssd_mobnet --pipeline_config_path=Documents/Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=5000


the command above will produce train folder and checkpoint file in model directory

# Evaluation
This cell will construct the command for doing evaluating the model.
The command can be runned in the terminal

In [ ]:
python Documents\Tensorflow\models\research\object_detection\model_main_tf2.py --model_dir=Documents\Tensorflow\workspace\models\my_ssd_mobnet --pipeline_config_path=Documents\Tensorflow\workspace\models\my_ssd_mobnet\pipeline.config --checkpoint_dir=Documents\Tensorflow\workspace\models\my_ssd_mobnet

the command above will produce eval folder in model directory

# Load Train Model From Checkpoint

In [11]:
import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

In [12]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-3')).expect_partial()

@tf.function()
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# Detect in Real-Time
In this section, the model will be tested using real-time camera from webcam source

In [14]:
import cv2 
import numpy as np

In [15]:
category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH+'/label_map.pbtxt')


In [16]:
# Setup capture
cap = cv2.VideoCapture(0) # 0 is represent source from computer webcam 
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [ ]:
while True: 
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=100,
                min_score_thresh=.8,
                agnostic_mode=False)

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        
        break

# Detecting from Image File
In this section, the model will be tested using image based input

In [12]:
import os

In [13]:
import cv2 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib tk

In [14]:
category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH+'/label_map.pbtxt')


In [16]:
IMAGE_PATH = 'Documents/Tensorflow/workspace/images/test/81.png'

In [17]:
img = cv2.imread(IMAGE_PATH)
image_np = np.array(img)

input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)

detections = detect_fn(input_tensor)

num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
              for key, value in detections.items()}
detections['num_detections'] = num_detections

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=10,
            min_score_thresh=0.25,
            agnostic_mode=False)

plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
plt.show()

# save model 

## 1. Freezing Graph

In [23]:
FREEZE_SCRIPT = 'Documents/Tensorflow/models/research/object_detection/exporter_main_v2.py'

In [26]:
command = "python {} --input_type=image_tensor --pipeline_config_path={} --trained_checkpoint_dir={} --output_directory={}".format(FREEZE_SCRIPT, CONFIG_PATH, 'Documents/Tensorflow/workspace/models/my_ssd_mobnet', 'Documents/Tensorflow/workspace/models/my_ssd_mobnet/export')

In [27]:
print(command)

python Documents/Tensorflow/models/research/object_detection/exporter_main_v2.py --input_type=image_tensor --pipeline_config_path=Documents/Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --trained_checkpoint_dir=Documents/Tensorflow/workspace/models/my_ssd_mobnet --output_directory=Documents/Tensorflow/workspace/models/my_ssd_mobnet/export


## 2. Converting to TFJS

In [40]:
!pip uninstall tensorflowjs -y

In [32]:
command = "tensorflowjs_converter --input_format=tf_saved_model --output_node_names='detection_boxes,detection_classes,detection_features,detection_multiclass_scores,detection_scores,num_detections,raw_detection_boxes,raw_detection_scores' --output_format=tfjs_graph_model --signature_name=serving_default {} {}".format(os.path.join('Documents/Tensorflow/workspace/models/my_ssd_mobnet/export', 'saved_model'), 'Documents/Tensorflow/workspace/models/my_ssd_mobnet/tfjsexport')

In [33]:
print(command)

tensorflowjs_converter --input_format=tf_saved_model --output_node_names='detection_boxes,detection_classes,detection_features,detection_multiclass_scores,detection_scores,num_detections,raw_detection_boxes,raw_detection_scores' --output_format=tfjs_graph_model --signature_name=serving_default Documents/Tensorflow/workspace/models/my_ssd_mobnet/export\saved_model Documents/Tensorflow/workspace/models/my_ssd_mobnet/tfjsexport


## 3. Converting to TFLite

In [42]:
TFLITE_SCRIPT = os.path.join('Documents/Tensorflow/models/research/object_detection/export_tflite_graph_tf2.py')

In [43]:
command = "python {} --pipeline_config_path={} --trained_checkpoint_dir={} --output_directory={}".format(TFLITE_SCRIPT ,CONFIG_PATH, 'Documents/Tensorflow/workspace/models/my_ssd_mobnet', 'Documents/Tensorflow/workspace/models/my_ssd_mobnet/tfliteexport')

In [44]:
print(command)

python Documents/Tensorflow/models/research/object_detection/export_tflite_graph_tf2.py --pipeline_config_path=Documents/Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --trained_checkpoint_dir=Documents/Tensorflow/workspace/models/my_ssd_mobnet --output_directory=Documents/Tensorflow/workspace/models/my_ssd_mobnet/tfliteexport


In [45]:
FROZEN_TFLITE_PATH = os.path.join('Documents/Tensorflow/workspace/models/my_ssd_mobnet/tfliteexport', 'saved_model')
TFLITE_MODEL = os.path.join('Documents/Tensorflow/workspace/models/my_ssd_mobnet/tfliteexport', 'saved_model', 'detect.tflite')

In [46]:
command = "tflite_convert \
--saved_model_dir={} \
--output_file={} \
--input_shapes=1,300,300,3 \
--input_arrays=normalized_input_image_tensor \
--output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' \
--inference_type=FLOAT \
--allow_custom_ops".format(FROZEN_TFLITE_PATH, TFLITE_MODEL, )

In [47]:
print(command)

tflite_convert --saved_model_dir=Documents/Tensorflow/workspace/models/my_ssd_mobnet/tfliteexport\saved_model --output_file=Documents/Tensorflow/workspace/models/my_ssd_mobnet/tfliteexport\saved_model\detect.tflite --input_shapes=1,300,300,3 --input_arrays=normalized_input_image_tensor --output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' --inference_type=FLOAT --allow_custom_ops
